In [2]:
import pandas as pd

df = pd.read_csv("../data/df_all_voi.csv")

print(df.columns)
print(df.head())
print(df.shape)


Index(['Identificativo noleggio', 'Targa veicolo', 'Data inizio corsa',
       'Data fine corsa', 'Lat inizio corsa_coordinate',
       'Lon inizio corsa_coordinate', 'Lat fine corsa_coordinate',
       'Lon fine corsa_coordinate', 'Tempo Tot', 'KM Tot', 'Prenotazione',
       'Batteria inizio', 'Batteria fine', 'source_file', 'start_hour',
       'start_date', 'weekday', 'is_weekend', 'duration_hours', 'speed_kmh',
       'trip_date', 'year', 'hour', 'month', 'week'],
      dtype='str')
                Identificativo noleggio Targa veicolo    Data inizio corsa  \
0  6dd0c93b-4557-4e8d-8f98-a543f24a761d          besc  2024-10-31 22:59:26   
1  201d1181-ad9a-4722-8754-280166181dbe          j2b6  2024-10-31 22:51:43   
2  201d1181-ad9a-4722-8754-280166181dbe          j2b6  2024-10-31 22:51:43   
3  9ac9534b-b86a-4c78-b5b0-39c9e3171227          yvld  2024-10-31 22:50:08   
4  d1caf896-84d3-4cb6-9d4a-03e538658dcc          9qxg  2024-10-31 22:47:52   

       Data fine corsa  Lat inizio cor

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/df_all_voi.csv")

# Rename important columns to simple names
df = df.rename(columns={
    "Data inizio corsa": "start_time",
    "Data fine corsa": "end_time",
    "Lat inizio corsa_coordinate": "start_lat",
    "Lon inizio corsa_coordinate": "start_lon",
    "Lat fine corsa_coordinate": "end_lat",
    "Lon fine corsa_coordinate": "end_lon",
    "Tempo Tot": "duration_raw",
    "KM Tot": "distance_km"
})

# Convert timestamps
df["start_time"] = pd.to_datetime(df["start_time"])
df["end_time"] = pd.to_datetime(df["end_time"])

# Duration in minutes (more reliable than raw field)
df["duration_min"] = (df["end_time"] - df["start_time"]).dt.total_seconds() / 60

# Remove bad data
before_clean = len(df)

df = df[
    (df["duration_min"] > 0) &
    (df["start_lat"].notna()) &
    (df["end_lat"].notna())
]

after_clean = len(df)

print("Records before cleaning:", before_clean)
print("Records after cleaning:", after_clean)

# Extract time variables
df["hour"] = df["start_time"].dt.hour
df["weekday"] = df["start_time"].dt.weekday
df["month"] = df["start_time"].dt.month

# Define peak periods
def peak_label(hour):
    if 7 <= hour <= 10:
        return "Morning Peak"
    elif 17 <= hour <= 20:
        return "Evening Peak"
    else:
        return "Off Peak"

df["period"] = df["hour"].apply(peak_label)

print(df.head())


Records before cleaning: 291148
Records after cleaning: 285025
                Identificativo noleggio Targa veicolo          start_time  \
0  6dd0c93b-4557-4e8d-8f98-a543f24a761d          besc 2024-10-31 22:59:26   
1  201d1181-ad9a-4722-8754-280166181dbe          j2b6 2024-10-31 22:51:43   
2  201d1181-ad9a-4722-8754-280166181dbe          j2b6 2024-10-31 22:51:43   
3  9ac9534b-b86a-4c78-b5b0-39c9e3171227          yvld 2024-10-31 22:50:08   
4  d1caf896-84d3-4cb6-9d4a-03e538658dcc          9qxg 2024-10-31 22:47:52   

             end_time  start_lat  start_lon    end_lat   end_lon  \
0 2024-10-31 23:01:19  45.043269   7.671645  45.048233  7.673718   
1 2024-10-31 22:59:10  45.074264   7.694650  45.086248  7.704112   
2 2024-10-31 22:59:10  45.074264   7.694650  45.086248  7.704112   
3 2024-10-31 23:04:01  45.076192   7.696033  45.086265  7.704420   
4 2024-10-31 23:03:58  45.074614   7.614285  45.058371  7.658567   

   duration_raw  distance_km  ... is_weekend  duration_hours  spe

In [4]:
df0 = pd.read_csv("../data/df_all_voi.csv")

print("Initial records:", len(df0))

# --- Rename ---
df0 = df0.rename(columns={
    "Data inizio corsa": "start_time",
    "Data fine corsa": "end_time",
    "Lat inizio corsa_coordinate": "start_lat",
    "Lon inizio corsa_coordinate": "start_lon",
    "Lat fine corsa_coordinate": "end_lat",
    "Lon fine corsa_coordinate": "end_lon",
    "Tempo Tot": "duration_raw",
    "KM Tot": "distance_km"
})

# --- Timestamp conversion ---
df0["start_time"] = pd.to_datetime(df0["start_time"], errors="coerce")
df0["end_time"] = pd.to_datetime(df0["end_time"], errors="coerce")

before_timestamp = len(df0)
df1 = df0.dropna(subset=["start_time", "end_time"])
after_timestamp = len(df1)

print("Removed due to invalid timestamps:", before_timestamp - after_timestamp)

# --- Duration calculation ---
df1["duration_min"] = (df1["end_time"] - df1["start_time"]).dt.total_seconds() / 60

before_duration = len(df1)
df2 = df1[df1["duration_min"] > 0]
after_duration = len(df2)

print("Removed due to non-positive duration:", before_duration - after_duration)

# --- Coordinate cleaning ---
before_coord = len(df2)
df3 = df2.dropna(subset=["start_lat", "start_lon", "end_lat", "end_lon"])
after_coord = len(df3)

print("Removed due to missing coordinates:", before_coord - after_coord)

print("Final cleaned records:", after_coord)


Initial records: 291148
Removed due to invalid timestamps: 6123
Removed due to non-positive duration: 0
Removed due to missing coordinates: 0
Final cleaned records: 285025


In [5]:
import pandas as pd

df_raw = pd.read_csv("../data/df_all_voi.csv")

# rename like before
df_raw = df_raw.rename(columns={
    "Data inizio corsa": "start_time",
    "Data fine corsa": "end_time"
})

# convert with coercion
df_raw["start_time_parsed"] = pd.to_datetime(df_raw["start_time"], errors="coerce")
df_raw["end_time_parsed"] = pd.to_datetime(df_raw["end_time"], errors="coerce")

# find invalid timestamps
invalid_time_rows = df_raw[
    df_raw["start_time_parsed"].isna() |
    df_raw["end_time_parsed"].isna()
]

print("Total invalid timestamp rows:", len(invalid_time_rows))

# show 10 examples
invalid_time_rows[[
    "start_time",
    "end_time"
]].head(10)


Total invalid timestamp rows: 6123


,start_time,end_time
117815,NaN,NaN
117816,NaN,NaN
117817,NaN,NaN
117818,NaN,NaN
117819,NaN,NaN
117820,NaN,NaN
117821,NaN,NaN
117822,NaN,NaN
117823,NaN,NaN
117824,NaN,NaN


In [6]:
import geopandas as gpd
from shapely.geometry import Point

# Load zoning shapefile
zones = gpd.read_file("../data/zone_statistiche_geo/zone_statistiche_geo.shp")

print(zones.crs)
print(zones.head())


EPSG:3003
  ZONASTAT                                        DENOM  \
0       70                                    Pilonetto   
1       42                             Borgata Vittoria   
2       55                            Ospizio di Carità   
3       06                       Piazza Vittorio Veneto   
4       09  Piazza Madama Cristina (Borgo San Salvario)   

                                            geometry  
0  POLYGON ((1395895.985 4988060.107, 1395898.044...  
1  POLYGON ((1395519.219 4994610.663, 1395528.682...  
2  POLYGON ((1392498.38 4988368.989, 1392593.575 ...  
3  POLYGON ((1396903.882 4991059.344, 1396929.822...  
4  POLYGON ((1395852.47 4990177.979, 1395864.654 ...  


In [7]:
# Fresh GeoDataFrame from cleaned df3
gdf = gpd.GeoDataFrame(
    df3.copy(),
    geometry=gpd.points_from_xy(df3["start_lon"], df3["start_lat"]),
    crs="EPSG:4326"
)

# Convert CRS
gdf = gdf.to_crs(zones.crs)

# ---- ORIGIN JOIN ----
origin_join = gpd.sjoin(
    gdf,
    zones[["ZONASTAT", "geometry"]],
    how="left",
    predicate="within"
)

origin_join = origin_join.rename(columns={"ZONASTAT": "origin_zone"})
origin_join = origin_join.drop(columns=["index_right"], errors="ignore")

# ---- DESTINATION JOIN ----
gdf_dest = gpd.GeoDataFrame(
    df3.copy(),
    geometry=gpd.points_from_xy(df3["end_lon"], df3["end_lat"]),
    crs="EPSG:4326"
)

gdf_dest = gdf_dest.to_crs(zones.crs)

dest_join = gpd.sjoin(
    gdf_dest,
    zones[["ZONASTAT", "geometry"]],
    how="left",
    predicate="within"
)

dest_join = dest_join.rename(columns={"ZONASTAT": "dest_zone"})
dest_join = dest_join.drop(columns=["index_right"], errors="ignore")

# ---- Combine cleanly by index ----
origin_join = origin_join.reset_index(drop=True)
dest_join = dest_join.reset_index(drop=True)

origin_join["dest_zone"] = dest_join["dest_zone"]

final_gdf = origin_join

print("Final trips:", len(final_gdf))
print("Missing origin zones:", final_gdf["origin_zone"].isna().sum())
print("Missing dest zones:", final_gdf["dest_zone"].isna().sum())


Final trips: 285025
Missing origin zones: 396
Missing dest zones: 512


In [8]:
final_gdf = final_gdf[
    final_gdf["origin_zone"].notna() &
    final_gdf["dest_zone"].notna()
]

print("Trips after removing missing zones:", len(final_gdf))


Trips after removing missing zones: 284174


In [9]:
od_matrix = (
    final_gdf
    .groupby(["origin_zone", "dest_zone"])
    .size()
    .reset_index(name="flow")
)

print(od_matrix.head())
print("Number of OD pairs:", len(od_matrix))


  origin_zone dest_zone  flow
0          01        01   998
1          01        02    82
2          01        03   287
3          01        04   361
4          01        05   426
Number of OD pairs: 4385


In [10]:
print("Columns in final_gdf:")
print(final_gdf.columns)


Columns in final_gdf:
Index(['Identificativo noleggio', 'Targa veicolo', 'start_time', 'end_time',
       'start_lat', 'start_lon', 'end_lat', 'end_lon', 'duration_raw',
       'distance_km', 'Prenotazione', 'Batteria inizio', 'Batteria fine',
       'source_file', 'start_hour', 'start_date', 'weekday', 'is_weekend',
       'duration_hours', 'speed_kmh', 'trip_date', 'year', 'hour', 'month',
       'week', 'duration_min', 'geometry', 'origin_zone', 'dest_zone'],
      dtype='str')


In [11]:
def peak_label(hour):
    if 7 <= hour <= 10:
        return "Morning Peak"
    elif 17 <= hour <= 20:
        return "Evening Peak"
    else:
        return "Off Peak"

final_gdf["period"] = final_gdf["hour"].apply(peak_label)

print(final_gdf["period"].value_counts())


period
Off Peak        176017
Evening Peak     62975
Morning Peak     45182
Name: count, dtype: int64


In [12]:
od_period = (
    final_gdf
    .groupby(["period", "origin_zone", "dest_zone"])
    .size()
    .reset_index(name="flow")
)

print(od_period.head())
print("Total OD-period combinations:", len(od_period))


         period origin_zone dest_zone  flow
0  Evening Peak          01        01   214
1  Evening Peak          01        02    13
2  Evening Peak          01        03    65
3  Evening Peak          01        04    83
4  Evening Peak          01        05   125
Total OD-period combinations: 9952


In [13]:
# Make sure zones CRS is correct
zones = zones.to_crs("EPSG:3003")

# Compute centroids
zones["centroid"] = zones.geometry.centroid

# Keep only needed columns
zone_centroids = zones[["ZONASTAT", "centroid"]].copy()

print(zone_centroids.head())


  ZONASTAT                         centroid
0       70  POINT (1396524.402 4988326.557)
1       42  POINT (1396007.137 4994520.119)
2       55  POINT (1393224.825 4988148.526)
3       06  POINT (1397230.097 4991149.509)
4       09  POINT (1396273.604 4990310.857)


In [14]:
from shapely.geometry import LineString
import geopandas as gpd

# Filter Evening Peak
evening_od = od_period[od_period["period"] == "Evening Peak"].copy()

# Merge origin centroid
evening_od = evening_od.merge(
    zone_centroids,
    left_on="origin_zone",
    right_on="ZONASTAT",
    how="left"
).rename(columns={"centroid": "origin_centroid"})

# Merge destination centroid
evening_od = evening_od.merge(
    zone_centroids,
    left_on="dest_zone",
    right_on="ZONASTAT",
    how="left"
).rename(columns={"centroid": "dest_centroid"})

# Create OD lines
evening_od["geometry"] = evening_od.apply(
    lambda row: LineString([row["origin_centroid"], row["dest_centroid"]]),
    axis=1
)

evening_gdf = gpd.GeoDataFrame(evening_od, geometry="geometry", crs=zones.crs)

print(evening_gdf.head())


         period origin_zone dest_zone  flow ZONASTAT_x  \
0  Evening Peak          01        01   214         01   
1  Evening Peak          01        02    13         01   
2  Evening Peak          01        03    65         01   
3  Evening Peak          01        04    83         01   
4  Evening Peak          01        05   125         01   

                   origin_centroid ZONASTAT_y  \
0  POINT (1396180.028 4991972.839)         01   
1  POINT (1396180.028 4991972.839)         02   
2  POINT (1396180.028 4991972.839)         03   
3  POINT (1396180.028 4991972.839)         04   
4  POINT (1396180.028 4991972.839)         05   

                     dest_centroid  \
0  POINT (1396180.028 4991972.839)   
1  POINT (1396772.228 4991841.696)   
2  POINT (1396673.463 4991230.712)   
3  POINT (1396090.766 4991148.058)   
4  POINT (1395603.966 4992283.328)   

                                            geometry  
0  LINESTRING (1396180.028 4991972.839, 1396180.0...  
1  LINESTRING (13

In [15]:
evening_top = evening_gdf.sort_values("flow", ascending=False).head(200)


In [16]:
evening_top = evening_top.to_crs("EPSG:4326")


In [17]:
import folium

m_evening = folium.Map(location=[45.07, 7.68], zoom_start=12)

for _, row in evening_top.iterrows():
    folium.GeoJson(
        row["geometry"],
        style_function=lambda x, weight=row["flow"]:
            {
                "color": "red",
                "weight": min(weight/50, 5),
                "opacity": 0.7
            }
    ).add_to(m_evening)

m_evening


In [18]:
m_evening.save("od_evening_flows_voi.html")

In [19]:
# Morning Peak
morning_od = od_period[od_period["period"] == "Morning Peak"].copy()

morning_od = morning_od.merge(
    zone_centroids,
    left_on="origin_zone",
    right_on="ZONASTAT",
    how="left"
).rename(columns={"centroid": "origin_centroid"})

morning_od = morning_od.merge(
    zone_centroids,
    left_on="dest_zone",
    right_on="ZONASTAT",
    how="left"
).rename(columns={"centroid": "dest_centroid"})

morning_od["geometry"] = morning_od.apply(
    lambda row: LineString([row["origin_centroid"], row["dest_centroid"]]),
    axis=1
)

morning_gdf = gpd.GeoDataFrame(morning_od, geometry="geometry", crs=zones.crs)

morning_top = morning_gdf.sort_values("flow", ascending=False).head(200)
morning_top = morning_top.to_crs("EPSG:4326")

m_morning = folium.Map(location=[45.07, 7.68], zoom_start=12)

for _, row in morning_top.iterrows():
    folium.GeoJson(
        row["geometry"],
        style_function=lambda x, weight=row["flow"]:
            {
                "color": "blue",
                "weight": min(weight/50, 5),
                "opacity": 0.7
            }
    ).add_to(m_morning)

m_morning


In [20]:
m_morning.save("od_morning_flows_voi.html")

In [21]:
evening_top = evening_gdf.sort_values("flow", ascending=False).head(200)
evening_top = evening_top.to_crs("EPSG:4326")

m_evening = folium.Map(location=[45.07, 7.68], zoom_start=12)

for _, row in evening_top.iterrows():
    folium.GeoJson(
        row["geometry"],
        style_function=lambda x, weight=row["flow"]:
            {
                "color": "red",
                "weight": min(weight/50, 5),
                "opacity": 0.7
            }
    ).add_to(m_evening)

m_evening


In [22]:
off_od = od_period[od_period["period"] == "Off Peak"].copy()

off_od = off_od.merge(
    zone_centroids,
    left_on="origin_zone",
    right_on="ZONASTAT",
    how="left"
).rename(columns={"centroid": "origin_centroid"})

off_od = off_od.merge(
    zone_centroids,
    left_on="dest_zone",
    right_on="ZONASTAT",
    how="left"
).rename(columns={"centroid": "dest_centroid"})

off_od["geometry"] = off_od.apply(
    lambda row: LineString([row["origin_centroid"], row["dest_centroid"]]),
    axis=1
)

off_gdf = gpd.GeoDataFrame(off_od, geometry="geometry", crs=zones.crs)

off_top = off_gdf.sort_values("flow", ascending=False).head(200)
off_top = off_top.to_crs("EPSG:4326")

m_off = folium.Map(location=[45.07, 7.68], zoom_start=12)

for _, row in off_top.iterrows():
    folium.GeoJson(
        row["geometry"],
        style_function=lambda x, weight=row["flow"]:
            {
                "color": "green",
                "weight": min(weight/50, 5),
                "opacity": 0.7
            }
    ).add_to(m_off)

m_off


In [23]:
m_off.save("od_off_flows_voi.html")

In [24]:
total_flows_by_period = (
    od_period
    .groupby("period")["flow"]
    .sum()
    .reset_index()
    .sort_values("flow", ascending=False)
)

total_flows_by_period


,period,flow
2,Off Peak,176017
0,Evening Peak,62975
1,Morning Peak,45182


In [25]:
# Top OD flows per period

top_od_by_period = (
    od_period
    .sort_values(["period", "flow"], ascending=[True, False])
    .groupby("period")
    .head(20)
    .reset_index(drop=True)
)

top_od_by_period


,period,origin_zone,dest_zone,flow
0,Evening Peak,59,35,1028
1,Evening Peak,16,16,1011
2,Evening Peak,35,35,782
3,Evening Peak,33,33,657
4,Evening Peak,50,50,522
5,Evening Peak,56,56,440
6,Evening Peak,08,26,414
7,Evening Peak,01,32,375
8,Evening Peak,35,33,371
9,Evening Peak,26,26,350


In [26]:
# Create lookup table for zone names
zone_lookup = zones[["ZONASTAT", "DENOM"]].copy()
zone_lookup = zone_lookup.rename(columns={
    "ZONASTAT": "zone_code",
    "DENOM": "zone_name"
})

zone_lookup.head()


,zone_code,zone_name
0,70,Pilonetto
1,42,Borgata Vittoria
2,55,Ospizio di Carità
3,06,Piazza Vittorio Veneto
4,09,Piazza Madama Cristina (Borgo San Salvario)


In [27]:
od_period_named = od_period.merge(
    zone_lookup,
    left_on="origin_zone",
    right_on="zone_code",
    how="left"
).rename(columns={"zone_name": "origin_name"})

od_period_named = od_period_named.drop(columns=["zone_code"])


In [28]:
od_period_named = od_period_named.merge(
    zone_lookup,
    left_on="dest_zone",
    right_on="zone_code",
    how="left"
).rename(columns={"zone_name": "dest_name"})

od_period_named = od_period_named.drop(columns=["zone_code"])


In [29]:
top_od_named = (
    od_period_named
    .sort_values(["period", "flow"], ascending=[True, False])
    .groupby("period")
    .head(20)
    .reset_index(drop=True)
)

top_od_named


,period,origin_zone,dest_zone,flow,origin_name,dest_name
0,Evening Peak,59,35,1028,Barriera di Orbassano,Polo Nord
1,Evening Peak,16,16,1011,Borgo San Donato,Borgo San Donato
2,Evening Peak,35,35,782,Polo Nord,Polo Nord
3,Evening Peak,33,33,657,Borgo San Paolo,Borgo San Paolo
4,Evening Peak,50,50,522,Borgata Parella - Lionetto,Borgata Parella - Lionetto
5,Evening Peak,56,56,440,Mercato Ortofrutticolo,Mercato Ortofrutticolo
6,Evening Peak,08,26,414,Comandi Militari - Stazione Porta Susa,Crocetta
7,Evening Peak,01,32,375,Municipio,Borgata Cenisia
8,Evening Peak,35,33,371,Polo Nord,Borgo San Paolo
9,Evening Peak,26,26,350,Crocetta,Crocetta


In [30]:
total_by_period = (
    od_period_named
    .groupby("period")["flow"]
    .sum()
    .reset_index(name="total_flow")
)

total_by_period


,period,total_flow
0,Evening Peak,62975
1,Morning Peak,45182
2,Off Peak,176017


In [31]:
top20_by_period = (
    top_od_named
    .groupby("period")["flow"]
    .sum()
    .reset_index(name="top20_flow")
)
top20_by_period


,period,top20_flow
0,Evening Peak,8937
1,Morning Peak,8231
2,Off Peak,26447


In [32]:
concentration = total_by_period.merge(
    top20_by_period,
    on="period"
)
concentration["top20_share_percent"] = (
    concentration["top20_flow"] /
    concentration["total_flow"] * 100
)
concentration.sort_values("top20_share_percent", ascending=False)


,period,total_flow,top20_flow,top20_share_percent
1,Morning Peak,45182,8231,18.217432
2,Off Peak,176017,26447,15.025253
0,Evening Peak,62975,8937,14.191346


In [33]:
od_period_named["flow_type"] = (
    od_period_named["origin_zone"] == od_period_named["dest_zone"]
)

od_period_named["flow_type"] = od_period_named["flow_type"].map({
    True: "Intra-zonal",
    False: "Inter-zonal"
})

od_period_named.head()


,period,origin_zone,dest_zone,flow,origin_name,dest_name,flow_type
0,Evening Peak,01,01,214,Municipio,Municipio,Intra-zonal
1,Evening Peak,01,02,13,Municipio,Palazzo Reale,Inter-zonal
2,Evening Peak,01,03,65,Municipio,Palazzo Carignano - Vecchio Ospedate (Borgo Nu...,Inter-zonal
3,Evening Peak,01,04,83,Municipio,Piazza San Carlo - Piazza Carlo Felice,Inter-zonal
4,Evening Peak,01,05,125,Municipio,Piazza Statuto,Inter-zonal


In [34]:
flow_structure = (
    od_period_named
    .groupby(["period", "flow_type"])["flow"]
    .sum()
    .reset_index()
)

flow_structure


,period,flow_type,flow
0,Evening Peak,Inter-zonal,51768
1,Evening Peak,Intra-zonal,11207
2,Morning Peak,Inter-zonal,37540
3,Morning Peak,Intra-zonal,7642
4,Off Peak,Inter-zonal,148040
5,Off Peak,Intra-zonal,27977


In [35]:
total_by_period = (
    flow_structure
    .groupby("period")["flow"]
    .sum()
    .reset_index(name="total_flow")
)

flow_structure = flow_structure.merge(
    total_by_period,
    on="period"
)

flow_structure["share_percent"] = (
    flow_structure["flow"] /
    flow_structure["total_flow"] * 100
)

flow_structure.sort_values(["period", "share_percent"], ascending=[True, False])


,period,flow_type,flow,total_flow,share_percent
0,Evening Peak,Inter-zonal,51768,62975,82.204049
1,Evening Peak,Intra-zonal,11207,62975,17.795951
2,Morning Peak,Inter-zonal,37540,45182,83.086185
3,Morning Peak,Intra-zonal,7642,45182,16.913815
4,Off Peak,Inter-zonal,148040,176017,84.105513
5,Off Peak,Intra-zonal,27977,176017,15.894487


In [36]:
# Aggregate total outgoing flows per zone and period
zone_outflows = (
    od_period_named
    .groupby(["period", "origin_zone", "origin_name"])["flow"]
    .sum()
    .reset_index()
)

zone_outflows.head()


,period,origin_zone,origin_name,flow
0,Evening Peak,01,Municipio,2533
1,Evening Peak,02,Palazzo Reale,146
2,Evening Peak,03,Palazzo Carignano - Vecchio Ospedate (Borgo Nu...,1086
3,Evening Peak,04,Piazza San Carlo - Piazza Carlo Felice,2087
4,Evening Peak,05,Piazza Statuto,1120


In [37]:
# Merge zone geometries with aggregated outflows
zone_outflows_map = zones.merge(
    zone_outflows,
    left_on="ZONASTAT",
    right_on="origin_zone",
    how="left"
)

zone_outflows_map.head()


,ZONASTAT,DENOM,geometry,centroid,period,origin_zone,origin_name,flow
0,70,Pilonetto,"POLYGON ((1395895.985 4988060.107, 1395898.044...",POINT (1396524.402 4988326.557),Evening Peak,70,Pilonetto,77.0
1,70,Pilonetto,"POLYGON ((1395895.985 4988060.107, 1395898.044...",POINT (1396524.402 4988326.557),Morning Peak,70,Pilonetto,51.0
2,70,Pilonetto,"POLYGON ((1395895.985 4988060.107, 1395898.044...",POINT (1396524.402 4988326.557),Off Peak,70,Pilonetto,253.0
3,42,Borgata Vittoria,"POLYGON ((1395519.219 4994610.663, 1395528.682...",POINT (1396007.137 4994520.119),Evening Peak,42,Borgata Vittoria,298.0
4,42,Borgata Vittoria,"POLYGON ((1395519.219 4994610.663, 1395528.682...",POINT (1396007.137 4994520.119),Morning Peak,42,Borgata Vittoria,201.0


In [38]:
import folium

evening_map_data = zone_outflows_map[
    zone_outflows_map["period"] == "Evening Peak"
].copy()

evening_map_data["flow"] = evening_map_data["flow"].fillna(0)

evening_map_data = evening_map_data.to_crs("EPSG:4326")

# 🔥 remove centroid column (important)
evening_map_data = evening_map_data.drop(columns=["centroid"], errors="ignore")

m_evening_choro = folium.Map(location=[45.07, 7.68], zoom_start=12)

folium.Choropleth(
    geo_data=evening_map_data,
    data=evening_map_data,
    columns=["ZONASTAT", "flow"],
    key_on="feature.properties.ZONASTAT",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Total Outgoing Trips (Evening Peak)"
).add_to(m_evening_choro)

m_evening_choro


In [47]:
m_evening_choro.save("(Evening Peak) Spatial Distribution of Outgoing Flows.html")

In [40]:
# Filter Morning Peak data
morning_map_data = zone_outflows_map[
    zone_outflows_map["period"] == "Morning Peak"
].copy()


In [45]:
import folium

m_morning_choro = folium.Map(location=[45.07, 7.68], zoom_start=12)

folium.Choropleth(
    geo_data=morning_map_data.drop(columns=["centroid"]),
    data=morning_map_data,
    columns=["ZONASTAT", "flow"],
    key_on="feature.properties.ZONASTAT",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Total Outgoing Trips (Morning Peak)"
).add_to(m_morning_choro)

m_morning_choro


In [48]:
m_morning_choro.save("(Morning Peak) Spatial Distribution of Outgoing Flows.html")

In [43]:
# Filter Off Peak data
off_map_data = zone_outflows_map[
    zone_outflows_map["period"] == "Off Peak"
].copy()


In [44]:
m_off_choro = folium.Map(location=[45.07, 7.68], zoom_start=12)

folium.Choropleth(
    geo_data=off_map_data.drop(columns=["centroid"]),
    data=off_map_data,
    columns=["ZONASTAT", "flow"],
    key_on="feature.properties.ZONASTAT",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Total Outgoing Trips (Off Peak)"
).add_to(m_off_choro)

m_off_choro


In [49]:
m_off_choro.save("(Off Peak) Spatial Distribution of Outgoing Flows.html")